#### Pipelines
- Output of one step serves as input of another step
- Pipelines makes it easy to apply same operations on train and test

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.drop(columns=['Name','PassengerId','Ticket','Cabin'], inplace=True)

In [4]:
df.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [108]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns=['Survived']),
    df['Survived'],
    test_size = 0.2,
    random_state=42
)

In [6]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [53]:
df.isna().value_counts()

Survived  Pclass  Sex    Age    SibSp  Parch  Fare   Embarked
False     False   False  False  False  False  False  False       712
                         True   False  False  False  False       177
                         False  False  False  False  True          2
Name: count, dtype: int64

In [7]:
trf1 = ColumnTransformer(
    transformers=[
        ('impute_age', SimpleImputer(), [2]),
        ('impute_embarked', SimpleImputer(strategy='most_frequent'),[6])
    ],
    remainder='passthrough'
)

trf2 = ColumnTransformer(
    transformers=[
        ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
    ],
    remainder='passthrough'
)

# after ohe on sex(sex_male) and embared(s, c, xQ) we will be having 10 columns
trf3 = ColumnTransformer(
    transformers=[
        ('scale', MinMaxScaler(), slice(0, 10))
        # ('scale', MinMaxScaler(), [0,1, 2, 3, 4, 5,6, 7, 8, 9])
    ]
)
trf4 = DecisionTreeClassifier()

#### Create Pipeline

In [8]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
])

In [9]:
### make_pipeline donot requires naming of steps
pipe = make_pipeline(trf1, trf2, trf3, trf4)

<img src="pic1.png"></img>

In [10]:
pipe.fit(x_train, y_train)
#pipe.fit_tranform used when jaab hamme koi model train na karna hoo
#pipe.fit agar model train karna hai then pipe.predict
#         agar model train nahi karna then pipe.transform


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [11]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [12]:
y_pred = pipe.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.6256983240223464

#### Exploring pipeline

In [15]:
pipe.named_steps['columntransformer-1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [20]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [26]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [47]:
pipe.named_steps['columntransformer-2'].transformers_[0][1].get_feature_names_out(['Sex', 'Embarked'])
#ohe on sex and embarked

array(['Sex_C', 'Sex_Q', 'Sex_S', 'Embarked_0.0', 'Embarked_4.0125',
       'Embarked_5.0', 'Embarked_6.2375', 'Embarked_6.4375',
       'Embarked_6.45', 'Embarked_6.4958', 'Embarked_6.75',
       'Embarked_6.8583', 'Embarked_6.95', 'Embarked_6.975',
       'Embarked_7.05', 'Embarked_7.0542', 'Embarked_7.125',
       'Embarked_7.1417', 'Embarked_7.225', 'Embarked_7.2292',
       'Embarked_7.25', 'Embarked_7.3125', 'Embarked_7.4958',
       'Embarked_7.5208', 'Embarked_7.55', 'Embarked_7.6292',
       'Embarked_7.65', 'Embarked_7.725', 'Embarked_7.7333',
       'Embarked_7.7375', 'Embarked_7.7417', 'Embarked_7.75',
       'Embarked_7.775', 'Embarked_7.7958', 'Embarked_7.8',
       'Embarked_7.8292', 'Embarked_7.8542', 'Embarked_7.8792',
       'Embarked_7.8875', 'Embarked_7.8958', 'Embarked_7.925',
       'Embarked_8.0292', 'Embarked_8.05', 'Embarked_8.1125',
       'Embarked_8.1375', 'Embarked_8.3', 'Embarked_8.3625',
       'Embarked_8.4042', 'Embarked_8.5167', 'Embarked_8.6542',
    

we are getting this error beacause we have used columns indices instead of columns names during transformations



In [66]:
x_train.head(
)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


✅ Correct & CLEAN solution (BEST PRACTICE)

👉 Use ONE ColumnTransformer

👉 Scale numeric columns by name

👉 Encode categorical columns separately

In [ ]:
preprocessing = ColumnTransformer(
    transformers=[
        ('impute_age', SimpleImputer(), ['Age']),
        ('impute_embarked', SimpleImputer(strategy='most_frequent'),['Embarked']),
        ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['Sex', 'Embarked']),
        # after ohe on sex(sex_male) and embared(s, c, xQ) we will be having 10 columns
        ('scale', MinMaxScaler(), ['Age', 'Fare', 'Pclass'])
        # ('scale', MinMaxScaler(), [0,1, 2, 3, 4, 5,6, 7, 8, 9])
    ],
    remainder='passthrough'
)
model = DecisionTreeClassifier()

- The above tranformations is incorrect because a single columns is used multiple times 
- Columns transformers actually works parallely so using a single column multiple times gives us anonomous results
- So we will be using multiple 

In [73]:
pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('model', model)
])

In [74]:
pipe.fit(x_train, y_train)

ValueError: could not convert string to float: 'S'

## ❗ Important Note: ColumnTransformer vs Pipeline (Very Common Pitfall)

### 🔴 Why your previous code failed

You faced this error:

```text
ValueError: could not convert string to float: 'S'
```

This happened **not because of sklearn**, but because of **how `ColumnTransformer` works**.

---

## 🔑 Key Concept (Must Remember)

> **`ColumnTransformer` works in PARALLEL, not sequentially.**

That means:

* All transformers inside a `ColumnTransformer` receive the **original input data**
* The output of one transformer is **NOT** passed to another

So this is ❌ **WRONG design**:

```python
ColumnTransformer([
    ('impute_embarked', SimpleImputer(), ['Embarked']),
    ('ohe_sex_embarked', OneHotEncoder(), ['Sex', 'Embarked'])
])
```

Because:

* `Embarked` is used **twice**
* Imputation does NOT happen before OHE

---

## 🚫 Common Mistakes (What NOT to do)

❌ Using the **same column in multiple transformers**

❌ Assuming ColumnTransformer applies steps one after another

❌ Using `remainder='passthrough'` when string columns are still present

❌ Scaling after OneHotEncoding using `slice()`

❌ Chaining multiple ColumnTransformers

---

## ✅ Golden Rule (Very Important)

> **If a column needs multiple operations → wrap them inside a `Pipeline`.**

* `Pipeline` → sequential (step-by-step)
* `ColumnTransformer` → parallel (column-wise)


## 🧠 One-line takeaway (Memorize this)

> **ColumnTransformer is parallel.
> Pipeline is sequential.
> For multiple steps on the same column → use a Pipeline inside ColumnTransformer.**

---

✅ This is the **correct mental model** for preprocessing in scikit-learn.


In [109]:
preprocessing = ColumnTransformer(
    transformers=[
        ('numerical_cols', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('scaler', MinMaxScaler()),
             ]),

        ['Age','Fare','Pclass']
        ),

        ('categorical_cols',
         Pipeline([
             ('imputer', SimpleImputer(strategy='most_frequent')),
             ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
         ]),
         ['Sex','Embarked']

        ) 

    ]
)


pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('model', DecisionTreeClassifier())
])

pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age', 'Fare', 'Pclass']),
                                                 ('categorical_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('model', DecisionTreeClassifier())])

In [110]:
pipe.named_steps

{'preprocessing': ColumnTransformer(transformers=[('numerical_cols',
                                  Pipeline(steps=[('imputer',
                                                   SimpleImputer(strategy='median')),
                                                  ('scaler', MinMaxScaler())]),
                                  ['Age', 'Fare', 'Pclass']),
                                 ('categorical_cols',
                                  Pipeline(steps=[('imputer',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('ohe',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse_output=False))]),
                                  ['Sex', 'Embarked'])]),
 'model': DecisionTreeClassifier()}

In [99]:
pipe.named_steps['preprocessing'].transformers_[1][1][1].categories_

[array(['female', 'male'], dtype=object), array(['C', 'Q', 'S'], dtype=object)]

In [111]:
pipe.named_steps['preprocessing'].transformers_[1][1][1].get_feature_names_out(['Sex', 'Embarked'])

array(['Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype=object)

In [112]:
y_pred=pipe.predict(x_test)

In [113]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.776536312849162